In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
import pandas as pd
import sqlite3

In [ ]:
# 데이터베이스 생성
conn = sqlite3.connect("etf_database.db")
cursor = conn.cursor()

In [ ]:
etfs = pd.read_csv("data/etf_list.csv", encoding="cp949")

etfs.head(3)

etfs.rename(columns={"수익률(최근 1년)" : "수익률_최근1년"},inplace=True)

In [ ]:
cursor.execute("""
    CREATE TABLE ETFs(
        종목코드 TEXT PRIMARY KEY,
        종목명 TEXT,
        상장일 TEXT,
        분류체계 TEXT,
        운용사 TEXT,
        수익률_최근1년 REAL,
        기초지수 TEXT,
        추적오차 REAL,
        순자산총액 REAL,
        괴리율 REAL,
        변동성 TEXT,
        복제방법 TEXT,
        총보수 REAL,
        과세유형 TEXT
        )
""")

In [ ]:
for idx,row in etfs.iterrows():
        cursor.execute(
            "INSERT INTO ETFs VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)",
            (
                str(row['종목코드']),
                str(row['종목명']),
                str(row['상장일']),
                str(row['분류체계']),
                str(row['운용사']),
                float(row['수익률_최근1년']) if not pd.isna(row['수익률_최근1년']) else None,
                str(row['기초지수']),
                float(row['추적오차'])  if not pd.isna(row['추적오차']) else None,
                float(row['순자산총액'])  if not pd.isna(row['순자산총액']) else None,
                str(row['괴리율']),
                str(row['변동성']),
                str(row['복제방법']),
                float(row['총보수'])  if not pd.isna(row['총보수']) else None,
                str(row['과세유형']),
            ))

conn.commit()

In [ ]:
cursor.execute("SELECT * FROM ETFs")

res = cursor.fetchall()

print(res)

In [ ]:
embedding = OllamaEmbeddings(model="bge-m3:latest")
vector_store = InMemoryVectorStore(
    embedding=embedding
)


_ = vector_store.add_texts(None)
retriever = vector_store.as_retriever(
    search_type = "mmr",
    search_kwargs = {'k' : 5}
)